In [1]:
import os, sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from data.data_helpers import get_dataframe

In [2]:
df = get_dataframe()
df.head()

c:\Coding\Language\Python\VirtualEnvs\cis5200\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Skipping existing file C:\Coding\Github-Repo\CIS-5200-Project\data\era5_tehachapi_2018_H1.nc
Skipping existing file C:\Coding\Github-Repo\CIS-5200-Project\data\era5_tehachapi_2018_H2.nc
Skipping existing file C:\Coding\Github-Repo\CIS-5200-Project\data\era5_tehachapi_2019_H1.nc
Skipping existing file C:\Coding\Github-Repo\CIS-5200-Project\data\era5_tehachapi_2019_H2.nc
Skipping existing file C:\Coding\Github-Repo\CIS-5200-Project\data\era5_tehachapi_2020_H1.nc
Skipping existing file C:\Coding\Github-Repo\CIS-5200-Project\data\era5_tehachapi_2020_H2.nc


,datetime,u10,v10,t2m,sp,hour_sin,hour_cos,month_sin,month_cos,doy_sin,doy_cos,wind_speed,target_next_hour
0,2018-01-01 00:00:00,-0.808231,-0.069685,291.890625,89925.2500,0.000000,1.000000,0.5,0.866025,0.017213,0.999852,0.811229,1.524293
1,2018-01-01 01:00:00,-1.340298,-0.725997,290.692627,89972.2500,0.258819,0.965926,0.5,0.866025,0.017213,0.999852,1.524293,1.245654
2,2018-01-01 02:00:00,-0.462882,-1.156458,288.544922,90036.1875,0.500000,0.866025,0.5,0.866025,0.017213,0.999852,1.245654,1.053555
3,2018-01-01 03:00:00,0.300817,-1.009697,285.121826,90091.3125,0.707107,0.707107,0.5,0.866025,0.017213,0.999852,1.053555,1.122294
4,2018-01-01 04:00:00,0.736362,-0.846944,283.715820,90125.1250,0.866025,0.500000,0.5,0.866025,0.017213,0.999852,1.122294,1.363163


In [3]:
X = df.drop(columns=['datetime', 'target_next_hour']).copy()
y = df['target_next_hour']

In [4]:
# ...existing code...
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Split data (keep time order)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Pipeline: scaler + MLP
model = make_pipeline(
    StandardScaler(),
    MLPRegressor(
        hidden_layer_sizes=(200, 100),
        activation="relu",
        solver="adam",
        early_stopping=False,
        validation_fraction=0.1,
        n_iter_no_change=10,
        max_iter=500,
        random_state=42,
        verbose=False
    )
)

# Train
model.fit(X_train, y_train)

# Predict & evaluate
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))
# ...existing code...

MSE: 0.10615136476061003
R²: 0.9400115895471047


In [9]:
# Transformer baseline using PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Train/test split without shuffling to respect temporal order
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Feature scaling (fit on train only)
scaler_tf = StandardScaler()
X_train_arr = scaler_tf.fit_transform(X_train_tf.values)
X_test_arr = scaler_tf.transform(X_test_tf.values)
y_train_arr = y_train_tf.values.astype(np.float32)
y_test_arr = y_test_tf.values.astype(np.float32)

# Torch tensors and data loaders
X_train_tensor = torch.tensor(X_train_arr, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_arr, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_arr, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test_arr, dtype=torch.float32).unsqueeze(1)
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

class TransformerRegressor(nn.Module):
    def __init__(self, n_features, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.feature_proj = nn.Linear(1, d_model)
        self.pos_enc = PositionalEncoding(d_model, n_features)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.head = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.feature_proj(x)
        x = self.pos_enc(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.head(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_tf = TransformerRegressor(n_features=X_train_tensor.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_tf.parameters(), lr=1e-3, weight_decay=1e-5)

best_val = float('inf')
patience = 8
wait = 0
n_epochs = 15
for epoch in range(1, n_epochs + 1):
    model_tf.train()
    train_losses = []
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        preds = model_tf(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    avg_train = float(np.mean(train_losses))

    model_tf.eval()
    val_losses = []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            preds = model_tf(xb)
            val_losses.append(criterion(preds, yb).item())
    avg_val = float(np.mean(val_losses)) if val_losses else float('nan')

    if avg_val < best_val - 1e-6:
        best_val = avg_val
        wait = 0
        os.makedirs("models/model5", exist_ok=True)
        torch.save({'model_state_dict': model_tf.state_dict(), 'scaler': scaler_tf}, 'models/model5/transformer_model.pt')
    else:
        wait += 1

    print(f'Epoch {epoch:03d} | Train Loss: {avg_train:.6f} | Val Loss: {avg_val:.6f} | Wait: {wait}/{patience}')
    if wait >= patience:
        print('Early stopping triggered.')
        break

# Final evaluation
model_tf.eval()
with torch.no_grad():
    preds = model_tf(X_test_tensor.to(device)).cpu().numpy().squeeze()

mse_tf = mean_squared_error(y_test_arr, preds)
r2_tf = r2_score(y_test_arr, preds)
print('Transformer Test MSE:', mse_tf)
print('Transformer Test R²:', r2_tf)

# Reload example:
# checkpoint = torch.load('models/model5/transformer_model.pt')
# model_tf.load_state_dict(checkpoint['model_state_dict'])
# scaler_loaded = checkpoint['scaler']

Epoch 001 | Train Loss: 0.934030 | Val Loss: 0.233371 | Wait: 0/8
Epoch 002 | Train Loss: 0.227914 | Val Loss: 0.168632 | Wait: 0/8
Epoch 002 | Train Loss: 0.227914 | Val Loss: 0.168632 | Wait: 0/8
Epoch 003 | Train Loss: 0.205514 | Val Loss: 0.166541 | Wait: 0/8
Epoch 003 | Train Loss: 0.205514 | Val Loss: 0.166541 | Wait: 0/8
Epoch 004 | Train Loss: 0.205116 | Val Loss: 0.156338 | Wait: 0/8
Epoch 004 | Train Loss: 0.205116 | Val Loss: 0.156338 | Wait: 0/8
Epoch 005 | Train Loss: 0.202219 | Val Loss: 0.180506 | Wait: 1/8
Epoch 005 | Train Loss: 0.202219 | Val Loss: 0.180506 | Wait: 1/8
Epoch 006 | Train Loss: 0.198540 | Val Loss: 0.168357 | Wait: 2/8
Epoch 006 | Train Loss: 0.198540 | Val Loss: 0.168357 | Wait: 2/8
Epoch 007 | Train Loss: 0.195995 | Val Loss: 0.161549 | Wait: 3/8
Epoch 007 | Train Loss: 0.195995 | Val Loss: 0.161549 | Wait: 3/8
Epoch 008 | Train Loss: 0.195029 | Val Loss: 0.182629 | Wait: 4/8
Epoch 008 | Train Loss: 0.195029 | Val Loss: 0.182629 | Wait: 4/8
Epoch 009 